# Desafío - Transformación y manipulación de datos (Parte II) - Maria Laura Oyarce

#### Para resolver este desafío deberás cargar datos desde Hojas de Excel del archivo US_Regional_Sales_Data.xlsx. En base a esto, debe generar una visualización de datos como la que se muestra en las hojas Reporte1 y Reporte2 del archivo reporte.xlsx. Para cumplir con lo solicitado, puedes aplicar los siguientes pasos:

1. Carga los datos de cada pestaña del archivo US_Regional_Sales_Data.xlsx en un DataFrame independiente. (hint: Puedes obtener los nombres de las hojas usando pd.ExcelFile(archivo).sheet_names)

In [1]:
import pandas as pd

#Primero obtenemos el nombre de las hojas
sheet_names = pd.ExcelFile('US_Regional_Sales_Data.xlsx').sheet_names
print('Hojas disponibles en el archivo:', sheet_names)

#Cargamos las hojas en distintos dataframes
dataframes = {sheet: pd.read_excel('US_Regional_Sales_Data.xlsx', sheet_name=sheet) for sheet in sheet_names}

#Veamos la cantidad de filas y columnas de cada hoja o dataframe
for sheet, df in dataframes.items():
    print(f'Hoja {sheet}, Tamaño: {df.shape}')
    print (df.columns)

Hojas disponibles en el archivo: ['Sales Orders Sheet', 'Customers Sheet', 'Store Locations Sheet', 'Products Sheet', 'Sales Team Sheet']
Hoja Sales Orders Sheet, Tamaño: (7991, 16)
Index(['OrderNumber', 'Sales Channel', 'WarehouseCode', 'ProcuredDate',
       'OrderDate', 'ShipDate', 'DeliveryDate', 'CurrencyCode', '_SalesTeamID',
       '_CustomerID', '_StoreID', '_ProductID', 'Order Quantity',
       'Discount Applied', 'Unit Price', 'Unit Cost'],
      dtype='object')
Hoja Customers Sheet, Tamaño: (50, 2)
Index(['_CustomerID', 'Customer Names'], dtype='object')
Hoja Store Locations Sheet, Tamaño: (367, 15)
Index(['_StoreID', 'City Name', 'County', 'StateCode', 'State', 'Type',
       'Latitude', 'Longitude', 'AreaCode', 'Population', 'Household Income',
       'Median Income', 'Land Area', 'Water Area', 'Time Zone'],
      dtype='object')
Hoja Products Sheet, Tamaño: (47, 2)
Index(['_ProductID', 'Product Name'], dtype='object')
Hoja Sales Team Sheet, Tamaño: (28, 3)
Index(['_SalesT

2. Cruza todos los DataFrames usando validación many_to_one, y guarda el resultado en una nueva variable llamada df_base.

In [2]:
#Cruzamos todos los dataframes para crear uno final

df_base = (dataframes['Sales Orders Sheet']
    .merge(dataframes['Customers Sheet'], on='_CustomerID', how='left')
    .merge(dataframes['Store Locations Sheet'], on='_StoreID', how='left')
    .merge(dataframes['Products Sheet'], on='_ProductID', how='left')
    .merge(dataframes['Sales Team Sheet'], on='_SalesTeamID', how='left'))

#Verificamos el tamaño y las columnas del dataframe resultante
print("Tamaño del DataFrame base:", df_base.shape)
print("Columnas del DataFrame base:", df_base.columns)

Tamaño del DataFrame base: (7991, 34)
Columnas del DataFrame base: Index(['OrderNumber', 'Sales Channel', 'WarehouseCode', 'ProcuredDate',
       'OrderDate', 'ShipDate', 'DeliveryDate', 'CurrencyCode', '_SalesTeamID',
       '_CustomerID', '_StoreID', '_ProductID', 'Order Quantity',
       'Discount Applied', 'Unit Price', 'Unit Cost', 'Customer Names',
       'City Name', 'County', 'StateCode', 'State', 'Type', 'Latitude',
       'Longitude', 'AreaCode', 'Population', 'Household Income',
       'Median Income', 'Land Area', 'Water Area', 'Time Zone', 'Product Name',
       'Sales Team', 'Region'],
      dtype='object')


3. Agrega las siguientes columnas a df_ base: hint: Asegúrate de que las columnas sean de tipo datetime.
 <img src='tabla1.png'>
 <img src='tabla2.png'>

In [3]:
#Nos aseguramos que las columnas sean de tipo datetime
date_columns = ['ProcuredDate', 'OrderDate', 'ShipDate', 'DeliveryDate']
for col in date_columns:
    df_base[col] = pd.to_datetime(df_base[col], errors='coerce')

#Creamos las columnas pedidas en el df_base
df_base['ProcurementDays'] = (df_base['OrderDate'] - df_base['ProcuredDate']).dt.days
df_base['ShippingDays'] = (df_base['ShipDate'] - df_base['OrderDate']).dt.days
df_base['DeliveryDays'] = (df_base['DeliveryDate'] - df_base['ShipDate']).dt.days
df_base['CustomerDays'] = df_base['ShippingDays'] + df_base['DeliveryDays']

#Mostramos las nuevas columnas para corroborar que se hizo bien el proceso
print(df_base[['ProcurementDays', 'ShippingDays', 'DeliveryDays', 'CustomerDays']].head())

   ProcurementDays  ShippingDays  DeliveryDays  CustomerDays
0              151            14             5            19
1              151            22            10            32
2              151            21            10            31
3              151             2             5             7
4               51            16            10            26


4. Agrega una nueva columna discreta al df_base, con el nombre CustomerDaysInterval que clasifique los valores de la columna CustomerDays en los siguientes intervalos:
- 0 to 15 days
- 15 to 30 days
- 30 to 45 days
- 45 to 60 days
- 60 to 75 days
- 75 to 90 days

hint: Utiliza pd.cut con estas etiquetas de Intervalo. Puedes utilizar la propiedad `.dt.days` para convertir la columna CustomerDays a valores numéricos. La propiedad `.dt.days` te permite obtener directamente la representación numérica de
la diferencia en días a partir de un objeto DateInterval.

In [4]:
#Creamos los intervalos y sus etiquetas
intervalos = [0, 15, 30, 45, 60, 75, 90]
etiquetas = ['0 to 15 days', '15 to 30 days', '30 to 45 days', '45 to 60 days',
             '60 to 75 days', '75 to 90 days']

#Creamos la columna 'CustomerDaysInterval'
df_base['CustomerDaysInterval'] = pd.cut(df_base['CustomerDays'], bins=intervalos, labels=etiquetas)

#Comprobamos que se haya hecho bien el proceso
print(df_base[['CustomerDays', 'CustomerDaysInterval']].head())

   CustomerDays CustomerDaysInterval
0            19        15 to 30 days
1            32        30 to 45 days
2            31        30 to 45 days
3             7         0 to 15 days
4            26        15 to 30 days


5. Utiliza una tabla pivote para mostrar el conteo de órdenes agrupando por Sales Team en las filas y CustomerDaysInterval en las columnas. Este reporte debe cuadrar con los datos de la pestaña Reporte1 del archivo reportes.xlsx

In [5]:
pivot_orders = pd.pivot_table(
    df_base, 
    index='Sales Team', 
    columns='CustomerDaysInterval', 
    values='OrderNumber',  #Contamos por 'OrderNumber' o cualquier identificador de orden
    aggfunc='count') #Contamos la cantidad de registros por combinación de Sales Team e intervalos

#Mostramos la tabla pivote
print(pivot_orders)

CustomerDaysInterval  0 to 15 days  15 to 30 days  30 to 45 days  \
Sales Team                                                         
Adam Hernandez                  81            179             42   
Anthony Berry                   76            181             41   
Anthony Torres                  81            162             45   
Carl Nguyen                     96            178             40   
Carlos Miller                   65            148             34   
Chris Armstrong                 92            156             44   
Donald Reynolds                 97            163             36   
Douglas Tucker                  87            145             45   
Frank Brown                     85            150             40   
George Lewis                   104            169             42   
Jerry Green                     85            165             46   
Joe Price                       87            138             41   
Jonathan Hawkins                81            14

Segun esta tabla pivote, ningun equipo tiene ordenes completadas en un tiempo mayor a 45 días y que el equipo de Todd Roberts es el equipo que completó más órdenes en menos de 15 días.

6. Agrega la siguiente columna calculada a la base:
<img src='tabla3.png'>

In [6]:
#Calculamos la columna GrossMargin
df_base['GrossMargin'] = df_base['Order Quantity'] * (
    df_base['Unit Price'] * (1 - df_base['Discount Applied']) - df_base['Unit Cost'])

#Verificamos el resultado
print(df_base[['Order Quantity', 'Unit Price', 'Discount Applied', 'Unit Cost', 'GrossMargin']].head())

   Order Quantity  Unit Price  Discount Applied  Unit Cost  GrossMargin
0               5      1963.1             0.075   1001.181    4073.4325
1               3      3939.6             0.075   3348.660     886.4100
2               1      1775.5             0.050    781.220     905.5050
3               8      2324.9             0.075   1464.687    5486.7640
4               8      1822.4             0.100   1476.144    1312.1280


7. En esta parte debes aplicar un porcentaje a la columna GrossMargin, pero ese porcentaje dependerá del intervalo en que se encuentre GrossMargin. hint: Utiliza pd.cut con labels numéricas para que la columna resultante sea de tipo float.
<img src='tabla4.png'>

In [7]:
#Definimos los intervalos y las etiquetas
intervalos_comision = [0, 100, 1000, 10000, 100000]
comisiones = [0.05, 0.10, 0.15, 0.20]

#Creamos la columna CommissionsPercentage
df_base['CommissionsPercentage'] = pd.cut(
    df_base['GrossMargin'], 
    bins=intervalos_comision, 
    labels=comisiones).astype(float) #nos aseguramos que sea flo

# Verificar el resultado
print(df_base[['GrossMargin', 'CommissionsPercentage']].head())

   GrossMargin  CommissionsPercentage
0    4073.4325                   0.15
1     886.4100                   0.10
2     905.5050                   0.10
3    5486.7640                   0.15
4    1312.1280                   0.15


8. Calcula el monto de la comisión, utilizando la Tabla del Anexo.
<img src='tabla5.png'>

In [8]:
#Creamos la columna CommissionsAmount
df_base['CommissionsAmount'] = df_base['GrossMargin'] * df_base['CommissionsPercentage']

#Verificamos los resultados
print(df_base[['GrossMargin', 'CommissionsPercentage', 'CommissionsAmount']].head())

   GrossMargin  CommissionsPercentage  CommissionsAmount
0    4073.4325                   0.15         611.014875
1     886.4100                   0.10          88.641000
2     905.5050                   0.10          90.550500
3    5486.7640                   0.15         823.014600
4    1312.1280                   0.15         196.819200


9. Calcula la comisión sobre el margen bruto.
<img src='tabla6.png'>

In [9]:
#Creamos la columna NetMargin
df_base['NetMargin'] = df_base['GrossMargin'] - df_base['CommissionsAmount']

#Verificamos los resultados
print(df_base[['GrossMargin', 'CommissionsAmount','NetMargin']].head())

   GrossMargin  CommissionsAmount    NetMargin
0    4073.4325         611.014875  3462.417625
1     886.4100          88.641000   797.769000
2     905.5050          90.550500   814.954500
3    5486.7640         823.014600  4663.749400
4    1312.1280         196.819200  1115.308800


10. Utilizando una tabla pivote, muestra la suma de los valores GrossMargin, NetMargin y CommissionAmount agrupando por Sales Team en las filas. Este reporte debe cuadrar con los datos de la pestaña Reporte2 del archivo reportes.xlsx

In [10]:
#Creamos la tabla
pivot_final = pd.pivot_table(
    df_base,
    index='Sales Team',  #Filas: Sales Team
    values=['GrossMargin', 'NetMargin', 'CommissionsAmount'],  #Columnas a sumar
    aggfunc='sum')  #Sumamos los valores

#Mostramos la tabla pivote
print(pivot_final)

                     CommissionsAmount  GrossMargin      NetMargin
Sales Team                                                        
Adam Hernandez           126637.207525  779095.1955  677987.332975
Anthony Berry            124061.181475  767943.9500  654798.341525
Anthony Torres           117962.211675  743901.4355  636758.048825
Carl Nguyen              123843.212050  774781.4675  661874.732450
Carlos Miller            110882.093875  668752.9290  580326.019125
Chris Armstrong          122565.707975  781608.7005  660857.419525
Donald Reynolds          143768.338700  885386.8095  754038.260800
Douglas Tucker           120806.403550  745528.5640  633786.054450
Frank Brown              104839.466050  670881.2510  570076.792950
George Lewis             136852.255325  839150.7460  710824.776675
Jerry Green              126660.436425  788587.7555  670379.034075
Joe Price                128822.930100  787707.4425  662662.106400
Jonathan Hawkins         126216.945000  781391.5870  664629.88

In [11]:
reporte2_df = pd.read_excel('reportes.xlsx', sheet_name='Reporte2')

print(reporte2_df.head())

       Sales Team  CommissionAmount  Gross Margin  Net Margin
0  Adam Hernandez         126637.21     779095.20   677987.33
1   Anthony Berry         124061.18     767943.95   654798.34
2  Anthony Torres         117962.21     743901.44   636758.05
3     Carl Nguyen         123843.21     774781.47   661874.73
4   Carlos Miller         110882.09     668752.93   580326.02


Finalmente nuestra tabla pivote coincide con el Reporte 2 del excel de reportes 🥳

Anexo
Tabla de comisiones
<img src='tabla7.png'>